In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

def get_playlist_tracks(playlist_id, client_id, client_secret):
    """
    Extract all tracks from a Spotify playlist with pagination support.
    
    Args:
        playlist_id: Spotify playlist ID or full URL
        client_id: Your Spotify Client ID
        client_secret: Your Spotify Client Secret
    
    Returns:
        pandas DataFrame with song names and artists
    """
    # Set up authentication
    client_credentials_manager = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
    # Extract playlist ID from URL if full URL is provided
    if 'spotify.com' in playlist_id:
        playlist_id = playlist_id.split('playlist/')[-1].split('?')[0]
    
    print(f"Fetching tracks from playlist: {playlist_id}")
    
    # Initialize list to store all tracks
    tracks_data = []
    
    # Get first batch of tracks
    results = sp.playlist_tracks(playlist_id)
    
    # Process all pages using pagination
    while results:
        # Extract track information from current batch
        for item in results['items']:
            if item['track'] is not None:  # Skip null tracks
                track = item['track']
                track_name = track['name']
                # Get all artists (songs can have multiple artists)
                artists = ', '.join([artist['name'] for artist in track['artists']])
                
                tracks_data.append({
                    'Song Name': track_name,
                    'Artist(s)': artists
                })
        
        # Check if there are more tracks to fetch
        if results['next']:
            results = sp.next(results)
            print(f"Fetched {len(tracks_data)} tracks so far...")
        else:
            break
    
    print(f"Total tracks fetched: {len(tracks_data)}")
    
    # Create DataFrame
    df = pd.DataFrame(tracks_data)
    return df


if __name__ == "__main__":
    # Your Spotify credentials
    CLIENT_ID = '0f5f408c76494254ac81a52f6f2b78f2'
    CLIENT_SECRET = '0a2e9fef88d4441bb8f19c2309fc3090'
    
    # Playlist ID or full URL
    PLAYLIST_ID = 'https://open.spotify.com/playlist/6i2Qd6OpeRBAzxfscNXeWp?si=c75919d9d12c434e'
    
    # Get tracks from playlist
    df = get_playlist_tracks(PLAYLIST_ID, CLIENT_ID, CLIENT_SECRET)
    
    # Display the DataFrame
    print("\n" + "="*50)
    print(df.to_string(index=True))
    
    # Save to CSV
    output_file = 'spotify_playlist_tracks.csv'
    df.to_csv(output_file, index=False)
    print(f"\nSaved to {output_file}")


4